In [11]:
from pathlib import Path
import h5py
import numpy as np
import tables  # enables reading BLOSC compression

## Full session loading

In [12]:
def load_split_eeg(root: Path) -> np.ndarray:
    """Load a full session of split EEG data.
    
    :param root: rhino root path
    :returns: full session EEG data
    
    """
    path = root.joinpath("protocols", "r1",
                         "subjects", "R1111M",
                         "experiments", "FR1",
                         "sessions", "0",
                         "ephys", "current_processed", "noreref")
    files = sorted(path.glob("*"))
    return np.array([np.fromfile(str(infile), dtype="int16") for infile in files])

In [13]:
def load_hdf5_eeg(path: Path) -> np.ndarray:
    """Load a full session of HDF5 EEG data.
    
    :param path: path to HDF5 file
    :returns: full session EEG data
    
    """
    with h5py.File(str(path), "r") as hfile:
        eeg = hfile["eeg"]
        arr = np.empty(eeg.shape, dtype=eeg.dtype)
        eeg.read_direct(arr)
        return arr
        # return hfile["eeg"][0, ...]

In [14]:
%%timeit
data = load_split_eeg(Path("/Users/depalati/mnt/rhino"))

The slowest run took 53.48 times longer than the fastest. This could mean that an intermediate result is being cached.
4.11 s ± 8.81 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit
# BLOSC compression (requires PyTables)
data = load_hdf5_eeg(Path("/Users/depalati/rhino_home/scratch/eeg_timeseries_blosc.h5"))

The slowest run took 17.09 times longer than the fastest. This could mean that an intermediate result is being cached.
1.2 s ± 2.03 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit
# no compression
data = load_hdf5_eeg(Path("/Users/depalati/rhino_home/scratch/eeg_timeseries_no_compression.h5"))

The slowest run took 13.83 times longer than the fastest. This could mean that an intermediate result is being cached.
815 ms ± 1.28 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%%timeit
# no chunking
data = load_hdf5_eeg(Path("/Users/depalati/rhino_home/scratch/no_chunks.h5"))

186 ms ± 13.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Loading epochs

In [145]:
def epochs_gen():
    for start in range(0, 1623000, 5000):
        yield start, start + 500

In [154]:
dur = 500
epochs = [(start, start + dur) for start in range(0, 1623000, 5000)]

In [155]:
def load_split_eeg_epochs(root, epochs) -> np.ndarray:
    """Load epochs from split EEG data.
    
    :param root:
    :returns: n_events x n_channels x time array
    
    """
    path = root.joinpath("protocols", "r1",
                         "subjects", "R1111M",
                         "experiments", "FR1",
                         "sessions", "0",
                         "ephys", "current_processed", "noreref")
    files = sorted(path.glob("*"))
    mmaps = [np.memmap(f, dtype="int16") for f in files]
    duration = epochs[0][1] - epochs[0][0]

    data = np.empty((len(epochs), len(files), duration), dtype="int16")
    
    for i, epoch in enumerate(epochs):
        data[i, :] = [mmap[epoch[0]:epoch[1]] for mmap in mmaps]
    return data

In [168]:
%%timeit
split_data = load_split_eeg_epochs(Path("/Users/depalati/mnt/rhino"), epochs)

376 ms ± 4.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [165]:
def load_hdf5_eeg_epochs(path, epochs) -> np.ndarray:
    """Load epochs from HDF5 EEG data.
    
    :param path: path to HDF5 file
    :returns: EEG data
    
    """
    with h5py.File(path, "r") as hfile:
        dset = hfile["eeg"]
        duration = epochs[0][1] - epochs[0][0]
        array = np.empty((len(epochs), dset.shape[1], duration), dtype=dset.dtype)
        for i, epoch in enumerate(epochs):
            array[i, :] = dset[0, :, epoch[0]:epoch[1]]
        return array

In [167]:
%%timeit
hdf_data = load_hdf5_eeg_epochs("/Users/depalati/rhino_home/scratch/no_chunks.h5", epochs)

420 ms ± 2.86 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [169]:
split_data = load_split_eeg_epochs(Path("/Users/depalati/mnt/rhino"), epochs)
hdf_data = load_hdf5_eeg_epochs("/Users/depalati/rhino_home/scratch/no_chunks.h5", epochs)

from numpy.testing import assert_equal
assert_equal(split_data, hdf_data)

In [171]:
%%timeit
_ = load_hdf5_eeg_epochs("/Users/depalati/rhino_home/scratch/eeg_timeseries_no_compression.h5", epochs)

459 ms ± 11.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [181]:
def load_hdf_eeg_epochs_from_memory(path, epochs) -> np.ndarray:
    with h5py.File(path, "r") as hfile:
        data = hfile["eeg"].value
    
    duration = epochs[0][1] - epochs[0][0]
    array = np.empty((len(epochs), data.shape[1], duration), dtype=data.dtype)
    
    for i, epoch in enumerate(epochs):
        array[i, :] = data[0, :, epoch[0]:epoch[1]]
    
    return array

In [182]:
%%timeit
_ = load_hdf_eeg_epochs_from_memory("/Users/depalati/rhino_home/scratch/no_chunks.h5", epochs)

215 ms ± 2.58 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Remarks

1. Loading an entire session of data from a contiguous HDF5 file is *significantly* faster than using split EEG files.
2. Loading epochs with HDF5 appears to be slightly worse than using memmaped split EEG files, but is of the same order of magnitude.
3. Using chunking in HDF5 gives significantly worse performance for reading an entire session and doesn't seem to help things when reading epochs.
4. Loading all data into memory before splitting into epochs is about twice as fast (for this dataset) as reading epochs directly from the file.

## Futher questions

1. Is there a performance difference if we use PyTables for reading instead of HDF5?
2. Can we get further benefits by changing how the data are laid out? For example, what if we use one array per channel instead of a single, large array?